In [1]:
from PIL import Image
import io
import numpy as np
from sklearn import model_selection

classes = ["nakamoto","jiro"]
num_classes = len(classes)
image_size = 50

In [ ]:
import google.cloud.storage as storage

bucket_name = '<your bucket name>'
bucket_path = 'gs://' + bucket_name + '/'

storage_client = storage.Client()

X = []
Y = []
for index, classlabel in enumerate(classes):
  bucket = storage_client.get_bucket(bucket_name)
  blobs = bucket.list_blobs(prefix=classlabel+'/')
  for blob in blobs:
    blob = bucket.get_blob(blob.name)
    file = blob.download_as_string()
    image = Image.open(io.BytesIO(file))
    image = image.convert("RGB")
    image = image.resize((image_size, image_size))
    data = np.asarray(image)
    X.append(data)
    Y.append(index)

X = np.array(X)
Y = np.array(Y)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y)

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import keras
import numpy as np

import google.cloud.storage as storage
bucket_name = '<your bucket name>'
bucket_path = 'gs://' + bucket_name + '/'

def copy_to_gcs(gcs_path, file_path):
  with file_io.FileIO(file_path, 'r') as reader:
     with file_io.FileIO(os.path.join(gcs_path, file_path), 'w+') as writer:
        writer.write(reader.read())

def model_train(X, y):
    model = Sequential()
    model.add(Conv2D(32,(3,3), padding='same',input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    #model.add(Dense(3))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    model.fit(X, y, batch_size=32, epochs=100)

    # モデルの保存
    model.save('./nakamoto_jiro_cnn.h5')
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob('nakamoto_jiro_cnn.h5')
    blob.upload_from_filename('nakamoto_jiro_cnn.h5')

    return model

def model_eval(model, X, y):
    scores = model.evaluate(X, y, verbose=1)
    print('Test Loss: ', scores[0])
    print('Test Accuracy: ', scores[1])

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.get_blob("data/X_train.data")
file = blob.download_as_string()


X_train = X_train.astype("float") / 256
X_test = X_test.astype("float") / 256
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

model = model_train(X_train, y_train)
model_eval(model, X_test, y_test)